In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib import pyplot
from scipy import stats
from scipy.signal import find_peaks, detrend
import pywt

from numpy import arange, array, linspace, loadtxt, log2, logspace, mean, polyfit
from numpy import zeros, pi, sin, cos, arctan2, sqrt, real, imag, conj, tile
from numpy import round, interp, diff, unique, where
from pandas import DataFrame, date_range
import matplotlib.dates as mdates
from matplotlib import pyplot
from scipy.stats import pearsonr, mannwhitneyu, kruskal, norm
from sklearn.linear_model import LinearRegression, Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GroupKFold
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,f1_score, recall_score, precision_score,roc_auc_score, roc_curve, auc,accuracy_score,classification_report,confusion_matrix
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier, GradientBoostingClassifier,AdaBoostClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from collections import defaultdict, Counter


import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import GlobalAveragePooling2D,GlobalMaxPooling2D,Concatenate,Input,Dropout, Conv2D, MaxPooling2D, Flatten,Multiply,Attention,Dense,concatenate,Masking,BatchNormalization, Reshape
from keras.models import Sequential
from keras.optimizers import Adam
from keras.applications import DenseNet121
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, History
history = History()

import pickle
import joblib
import dill
import json
from collections import defaultdict
from scipy.signal import savgol_filter

import warnings
warnings.filterwarnings('ignore')

# os.environ["CUDA_VISIBLE_DEVICES"] = "2,3,4,5"


2024-11-25 12:16:06.891035: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-25 12:16:07.012296: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-25 12:16:07.012326: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-25 12:16:07.012334: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 12:16:07.035990: I tensorflow/core/platform/cpu_feature_g

In [2]:
DEXI_folder = '/mnt/data1/ma98/Compressed_DEXI_Data/'

T1DEXI_CGM_Dataset = pd.read_parquet(os.path.join(DEXI_folder, 'T1DEXI_Dataset.csv')) 
T1DEXI_CGM_Dataset['LBDTC'] = pd.to_datetime(T1DEXI_CGM_Dataset['LBDTC'])

T1DEXI_Insulin_Dataset = pd.read_parquet(os.path.join(DEXI_folder, 'T1DEXI_Insulin_Dataset.csv'))  
T1DEXI_Insulin_Dataset['FADTC'] = pd.to_datetime(T1DEXI_Insulin_Dataset['FADTC'], format='%Y-%m-%d %H:%M:%S')

T1DEXI_Reqcue_Carbs_Dataset = pd.read_parquet(os.path.join(DEXI_folder, 'T1DEXI_Reqcue_Carbs_Dataset.csv'))  
T1DEXI_Reqcue_Carbs_Dataset['MLDTC'] = pd.to_datetime(T1DEXI_Reqcue_Carbs_Dataset['MLDTC'])

FAMLPM_Dataset = pd.read_parquet(os.path.join(DEXI_folder, 'FAMLPM_Dataset.csv')) 
FAMLPM_Dataset['FADTC'] = pd.to_datetime(FAMLPM_Dataset['FADTC'], format='%Y-%m-%d %H:%M:%S')

T1DEXI_sleep_Dataset = pd.read_parquet(os.path.join(DEXI_folder, 'T1DEXI_sleep_Dataset.csv')) 

T1DEXI_VS_Dataset = pd.read_csv(os.path.join(DEXI_folder, 'T1DEXI_VS_Dataset.csv.gz'))  
T1DEXI_VS_Dataset['VSDTC'] = pd.to_datetime(T1DEXI_VS_Dataset['VSDTC'], format='%Y-%m-%d %H:%M:%S')

#T1DEXI_basal = T1DEXI_Insulin_Dataset[T1DEXI_Insulin_Dataset['INSDVSRC'] != "Injections"]
T1DEXI_basal = T1DEXI_Insulin_Dataset.copy()
shared_participants = sorted(set(T1DEXI_CGM_Dataset['USUBJID']) & set(T1DEXI_basal['USUBJID'])& set(T1DEXI_Reqcue_Carbs_Dataset['USUBJID'])& 
                             set(FAMLPM_Dataset['USUBJID']) & set(T1DEXI_VS_Dataset['USUBJID'])) 

print(len(shared_participants))


493


In [8]:
CGM_data = T1DEXI_CGM_Dataset[T1DEXI_CGM_Dataset['LBTEST'] == "Glucose"].dropna(subset=['LBSTRESN']).reset_index()['LBSTRESN']
missing_values = CGM_data.isna().sum()
print(f"Number of missing values: {missing_values}")

average = CGM_data.mean()
std_dev = CGM_data.std()
print(f"Average: {average}")
print(f"Standard Deviation: {std_dev}")

threshold = 70
below_threshold_count = (CGM_data < threshold).sum()
non_missing_count = len(CGM_data) - missing_values
percentage_below_threshold = (below_threshold_count / non_missing_count) * 100
print(f"Percentage of data below {threshold}: {percentage_below_threshold:.2f}%")


Number of missing values: 0
Average: 146.14627450265544
Standard Deviation: 56.38279796326335
Percentage of data below 70: 3.08%


In [ ]:
data_dict = defaultdict(lambda: defaultdict(dict))

for participant_id in shared_participants: 
    participant_cgm_data = T1DEXI_CGM_Dataset[T1DEXI_CGM_Dataset['USUBJID'] == participant_id].reset_index()

    a1c_data = participant_cgm_data[participant_cgm_data['LBTEST'] == "Hemoglobin A1c"]
    a1c = a1c_data['LBSTRESN'].iloc[0]

    participant_cgm_data = participant_cgm_data[participant_cgm_data['LBTEST'] == "Glucose"].dropna(subset=['LBSTRESN']).reset_index()

    aligned_cgm = resample_avg(participant_cgm_data['LBSTRESN'], participant_cgm_data['LBDTC'], interval='5T')
    



In [3]:
def cgm_prediction_stats(current_day_cgm):
    general_stats = calculate_stats(current_day_cgm.values[:,0])
    general_stats = extract_prediction_stats(general_stats, 'cgm')

    masks = {
        'night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 6),
        'early_night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 3),
        'late_night': (current_day_cgm.index.hour >= 3) & (current_day_cgm.index.hour < 6),
        'long_night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 8),
        'night_morning': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 12),
        'morning': (current_day_cgm.index.hour >= 6) & (current_day_cgm.index.hour < 12),
        'afternoon': (current_day_cgm.index.hour >= 12) & (current_day_cgm.index.hour < 18),
        'evening': (current_day_cgm.index.hour >= 18) & (current_day_cgm.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(current_day_cgm, mask)
        period_stats = extract_prediction_stats(period_stats, f'cgm_{period}')
        time_range_stats.update(period_stats)
    
    time_in_range_stats = {
        'time_in_range_70_140_day': calculate_time_in_range(current_day_cgm, 70, 140),
        'time_in_range_70_180_day': calculate_time_in_range(current_day_cgm, 70, 180),
        'time_below_range_70_day': calculate_time_in_range(current_day_cgm, 0, 70),
        'time_above_range_140_day': calculate_time_in_range(current_day_cgm, 140, np.inf),
        'time_above_range_180_day': calculate_time_in_range(current_day_cgm, 180, np.inf)
    }
    for period, mask in masks.items():
        time_in_range_stats[f'time_in_range_70_140_{period}'] = calculate_time_in_range(current_day_cgm, 70, 140, mask)
        time_in_range_stats[f'time_in_range_70_180_{period}'] = calculate_time_in_range(current_day_cgm, 70, 180, mask)
        time_in_range_stats[f'time_below_range_70_{period}'] = calculate_time_in_range(current_day_cgm, 0, 70, mask)
        time_in_range_stats[f'time_above_range_140_{period}'] = calculate_time_in_range(current_day_cgm, 140, np.inf, mask)
        time_in_range_stats[f'time_above_range_180_{period}'] = calculate_time_in_range(current_day_cgm, 180, np.inf, mask)
    
    cgm_prediction_stats = pd.DataFrame({**general_stats, **time_range_stats, **time_in_range_stats})
    return cgm_prediction_stats

def get_daily_cgm_stats(current_day_cgm):
    general_stats = calculate_stats(current_day_cgm.values[:,0])
    general_stats = extract_stats(general_stats, 'daily_cgm')
    mean_crossings = calculate_crossings(current_day_cgm.values[:,0])
    general_stats['daily_cgm_mean_crossings'] = mean_crossings

    # Add new features
    last_cgm_value = current_day_cgm.values[-1, 0]
    general_stats['last_cgm_value'] = last_cgm_value
    general_stats['last_cgm_sq'] = np.sqrt(last_cgm_value)
    
    diff_10_min = last_cgm_value - current_day_cgm.values[-3, 0]
    general_stats['diff_10_min'] = diff_10_min

    diff_20_min = last_cgm_value - current_day_cgm.values[-5, 0]
    general_stats['diff_20_min'] = diff_20_min

    diff_30_min = last_cgm_value - current_day_cgm.values[-7, 0]
    general_stats['diff_30_min'] = diff_30_min

    diff_50_min = last_cgm_value - current_day_cgm.values[-11, 0]
    general_stats['diff_50_min'] = diff_50_min
    
    # Rate of change (slope) in CGM in the last hours
    slope_last_half_hour = (last_cgm_value - current_day_cgm.values[-7, 0]) / last_cgm_value
    general_stats['slope_last_half_hour'] = slope_last_half_hour
    slope_last_hour = (last_cgm_value - current_day_cgm.values[-13, 0]) / last_cgm_value
    general_stats['slope_last_hour'] = slope_last_hour
    slope_last_2_hour = (last_cgm_value - current_day_cgm.values[-25, 0]) / last_cgm_value
    general_stats['slope_last_2_hour'] = slope_last_2_hour
    
    # SD of CGM in the last hours
    std_last_2_hours = np.std(current_day_cgm.values[-25:, 0])
    general_stats['std_last_2_hours'] = std_last_2_hours
    std_last_hour = np.std(current_day_cgm.values[-13:, 0])
    general_stats['std_last_hour'] = std_last_hour
    
    # Sum of all increments in adjacent CGM observations in the last two hours
    increments_sum_last_2_hours = np.sum(np.diff(current_day_cgm.values[-25:, 0])[np.diff(current_day_cgm.values[-25:, 0]) > 0])
    general_stats['increments_sum_last_2_hours'] = increments_sum_last_2_hours

    # Sum of all decrements in adjacent CGM observations in the last two hours
    decrements_sum_last_2_hours = np.sum(np.diff(current_day_cgm.values[-25:, 0])[np.diff(current_day_cgm.values[-25:, 0]) < 0])
    general_stats['decrements_sum_last_2_hours'] = decrements_sum_last_2_hours

    # Maximum increase in adjacent CGM observations in the last two hours
    max_increase_last_2_hours = np.max(np.diff(current_day_cgm.values[-25:, 0]))
    general_stats['max_increase_last_2_hours'] = max_increase_last_2_hours

    # Maximum decrease in adjacent CGM observations in the last two hours
    max_decrease_last_2_hours = np.min(np.diff(current_day_cgm.values[-25:, 0]))
    general_stats['max_decrease_last_2_hours'] = max_decrease_last_2_hours

    masks = {
        'night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 6),
        'morning': (current_day_cgm.index.hour >= 6) & (current_day_cgm.index.hour < 12),
        'afternoon': (current_day_cgm.index.hour >= 12) & (current_day_cgm.index.hour < 18),
        'evening': (current_day_cgm.index.hour >= 18) & (current_day_cgm.index.hour < 24),
        'late_evening': (current_day_cgm.index.hour >= 21) & (current_day_cgm.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(current_day_cgm, mask)
        period_stats = extract_stats(period_stats, f'daily_cgm_{period}')
        time_range_stats.update(period_stats)
    
    time_in_range_stats = {
        'daily_time_in_range_70_140_day': calculate_time_in_range(current_day_cgm, 70, 140),
        'daily_time_in_range_70_180_day': calculate_time_in_range(current_day_cgm, 70, 180),
        'daily_time_below_range_70_day': calculate_time_in_range(current_day_cgm, 0, 70),
        'daily_time_above_range_140_day': calculate_time_in_range(current_day_cgm, 140, np.inf),
        'daily_time_above_range_180_day': calculate_time_in_range(current_day_cgm, 180, np.inf)
    }
    for period, mask in masks.items():
        time_in_range_stats[f'daily_time_in_range_70_140_{period}'] = calculate_time_in_range(current_day_cgm, 70, 140, mask)
        time_in_range_stats[f'daily_time_in_range_70_180_{period}'] = calculate_time_in_range(current_day_cgm, 70, 180, mask)
        time_in_range_stats[f'daily_time_below_range_70_{period}'] = calculate_time_in_range(current_day_cgm, 0, 70, mask)
        time_in_range_stats[f'daily_time_above_range_140_{period}'] = calculate_time_in_range(current_day_cgm, 140, np.inf, mask)
        time_in_range_stats[f'daily_time_above_range_180_{period}'] = calculate_time_in_range(current_day_cgm, 180, np.inf, mask)
    
    daily_cgm_stats = pd.DataFrame({**general_stats, **time_range_stats, **time_in_range_stats})
    return daily_cgm_stats

def get_total_cgm_stats(current_day_cgm):
    general_stats = calculate_stats(current_day_cgm.values[:,0])
    general_stats = extract_stats(general_stats, 'total_cgm')
    mean_crossings = calculate_crossings(current_day_cgm.values[:,0])
    general_stats['cgm_mean_crossings'] = mean_crossings

    masks = {
        'night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 6),
        'early_night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 3),
        'late_night': (current_day_cgm.index.hour >= 3) & (current_day_cgm.index.hour < 6),
        'morning': (current_day_cgm.index.hour >= 6) & (current_day_cgm.index.hour < 12),
        'afternoon': (current_day_cgm.index.hour >= 12) & (current_day_cgm.index.hour < 18),
        'evening': (current_day_cgm.index.hour >= 18) & (current_day_cgm.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(current_day_cgm, mask)
        period_stats = extract_stats(period_stats, f'total_cgm_{period}')
        time_range_stats.update(period_stats)
    
    time_in_range_stats = {
        'total_time_in_range_70_140_day': calculate_time_in_range(current_day_cgm, 70, 140),
        'total_time_in_range_70_170_day': calculate_time_in_range(current_day_cgm, 70, 180),
        'total_time_below_range_80_day': calculate_time_in_range(current_day_cgm, 0, 70),
        'total_time_above_range_140_day': calculate_time_in_range(current_day_cgm, 140, np.inf),
        'total_time_above_range_180_day': calculate_time_in_range(current_day_cgm, 180, np.inf)
    }
    for period, mask in masks.items():
        time_in_range_stats[f'total_time_in_range_70_140_{period}'] = calculate_time_in_range(current_day_cgm, 70, 140, mask)
        time_in_range_stats[f'total_time_in_range_70_180_{period}'] = calculate_time_in_range(current_day_cgm, 70, 180, mask)
        time_in_range_stats[f'total_time_below_range_70_{period}'] = calculate_time_in_range(current_day_cgm, 0, 70, mask)
        time_in_range_stats[f'total_time_above_range_140_{period}'] = calculate_time_in_range(current_day_cgm, 140, np.inf, mask)
        time_in_range_stats[f'total_time_above_range_180_{period}'] = calculate_time_in_range(current_day_cgm, 180, np.inf, mask)
    
    total_cgm_stats = pd.DataFrame({**general_stats, **time_range_stats, **time_in_range_stats})
    return total_cgm_stats

def daily_cgm_labels(current_day_cgm):
    masks = {
        'night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 6),
        'early_night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 3),
        'late_night': (current_day_cgm.index.hour >= 3) & (current_day_cgm.index.hour < 6),
        'long_night': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 8),
        'night_morning': (current_day_cgm.index.hour >= 0) & (current_day_cgm.index.hour < 12),
        'morning': (current_day_cgm.index.hour >= 6) & (current_day_cgm.index.hour < 12),
        'afternoon': (current_day_cgm.index.hour >= 12) & (current_day_cgm.index.hour < 18),
        'evening': (current_day_cgm.index.hour >= 18) & (current_day_cgm.index.hour < 24)
    }
    labels = {
        'hyper_day': return_label(current_day_cgm.values[:,0], 180),
        'hypo_day': return_label(current_day_cgm.values[:,0], 70)
    }
    for period, mask in masks.items():
        labels[f'hyper_{period}'] = return_label(current_day_cgm.values[:,0], 180, mask)
        labels[f'hypo_{period}'] = return_label(current_day_cgm.values[:,0], 70, mask)
    
    daily_cgm_labels = pd.DataFrame({**labels}, index=[0])
    return daily_cgm_labels

# --- Calculate Time in Range for Different Ranges and Periods ---
def calculate_time_in_range(glucose_data, range_lower, range_upper, period_mask=None):

    if period_mask is not None:
        glucose_data = glucose_data[period_mask]
    in_range = (glucose_data <= range_upper) & (glucose_data >= range_lower)
    return 100 * in_range.sum() / len(glucose_data) if len(glucose_data) > 0 else 0

def return_label(glucose_data, range, period_mask=None):
    if period_mask is not None:
        glucose_data = glucose_data[period_mask]
    if range==180:
        return np.sum(glucose_data>range)>2
    if range==70:
        return np.sum(glucose_data<range)>2
        
def calculate_sleep_statistics(daily_sleep_data):
    
    participant_total_sleep = participant_sleep_data[participant_sleep_data['NVTEST'] == "Total Sleep Time"]
    participant_deep_sleep = participant_sleep_data[participant_sleep_data['NVTEST'] == "Deep NREM Duration"]
    participant_light_sleep = participant_sleep_data[participant_sleep_data['NVTEST'] == "Light NREM Duration"]
    participant_NREM_sleep = participant_sleep_data[participant_sleep_data['NVTEST'] == "NREM Duration"]
    participant_REM_sleep = participant_sleep_data[participant_sleep_data['NVTEST'] == "REM Duration"]
    participant_efficiency = participant_sleep_data[participant_sleep_data['NVTEST'] == "Sleep Efficiency"]
    participant_awakenings = participant_sleep_data[participant_sleep_data['NVTEST'] == "Number of Awakenings"]
    participant_latency = participant_sleep_data[participant_sleep_data['NVTEST'] == "Sleep Onset Latency"]

    avg_total_sleep = (participant_total_sleep['NVORRES']).mean()
    std_total_sleep = (participant_total_sleep['NVORRES']).std()
    avg_deep_sleep = (participant_deep_sleep['NVORRES']).mean()
    std_deep_sleep = (participant_deep_sleep['NVORRES']).std()
    avg_light_sleep = (participant_light_sleep['NVORRES']).mean()
    std_light_sleep = (participant_light_sleep['NVORRES']).std()
    avg_NREM_sleep = (participant_NREM_sleep['NVORRES']).mean()
    std_NREM_sleep = (participant_NREM_sleep['NVORRES']).std()
    avg_REM_sleep = (participant_REM_sleep['NVORRES']).mean()
    std_REM_sleep = (participant_REM_sleep['NVORRES']).std()
    avg_awakenings = (participant_awakenings['NVORRES']*100000.).mean()
    std_awakenings = (participant_awakenings['NVORRES']*100000. ).std()
    avg_latency = (participant_latency['NVORRES']).mean()
    std_latency = (participant_latency['NVORRES']).std()
    avg_efficiency = (participant_efficiency['NVORRES']*100000.).mean()
    std_efficiency = (participant_efficiency['NVORRES']*100000.).std()
    
    # Calculate avg deviation from midnight
    bedtime_deviations_midnight = []
    wakeup_deviations_midnight = []
    for dt in participant_total_sleep['NVDTC']:
        sleep_time = 60*dt.hour + (dt.minute)
        
        if sleep_time >720:
            sleep_time -= 1440
        bedtime_deviations_midnight.append(sleep_time)
    
    for dt in participant_total_sleep['NVENDTC']:
        wakeup_time = 60*dt.hour + (dt.minute)
        wakeup_deviations_midnight.append(wakeup_time)
  
    average_bedtime_midnight = np.mean(bedtime_deviations_midnight)
    average_wakeup_midnight = np.mean(wakeup_deviations_midnight)
    std_bedtime_midnight = np.std(bedtime_deviations_midnight)
    std_wakeup_midnight =  np.std(wakeup_deviations_midnight)
   #     # Convert average deviation back to time format
    average_bedtime = pd.to_datetime('00:00:00') + pd.to_timedelta(average_bedtime_midnight, unit='m')
    average_wakeup = pd.to_datetime('00:00:00') + pd.to_timedelta(average_wakeup_midnight, unit='m')
       
    # Calculate bedtime deviations from average & variance & Consistency Score (inverse of variance)
    participant_sleep_data['bedtime_from_avg'] = ((60*participant_sleep_data['NVDTC'].dt.hour + participant_sleep_data['NVDTC'].dt.minute) - average_bedtime_midnight)
    participant_sleep_data.loc[participant_sleep_data['bedtime_from_avg'] > 720, 'bedtime_from_avg'] -= 1440

    participant_sleep_data['wakeup_from_avg'] = ((60*participant_sleep_data['NVENDTC'].dt.hour + participant_sleep_data['NVENDTC'].dt.minute) - average_wakeup_midnight)
    bedtime_std = np.round(participant_sleep_data['bedtime_from_avg'].std(),3)
    wakeup_std = np.round(participant_sleep_data['wakeup_from_avg'].std(),3)
    bedtime_var = np.round(participant_sleep_data['bedtime_from_avg'].var(),3)
    wakeup_var = np.round(participant_sleep_data['wakeup_from_avg'].var(),3)
    sleep_stats = pd.DataFrame({
        'avg_bedtime_midnight': [average_bedtime_midnight.round(3)],
        'avg_bedtime': [average_bedtime],
        'bedtime_consistency_score': [(100. / bedtime_var).round(3)],
        'bedtime_std': [bedtime_std],
        'bedtime_var': [bedtime_var],
        'avg_wakeup_midnight': [average_wakeup_midnight.round(3)],
        'avg_wakeup': [average_wakeup],
        'wakeup_consistency_score': [(100. / wakeup_var).round(3)],
        'wakeup_std': [wakeup_std],
        'wakeup_var': [wakeup_var],
        'avg_deep_sleep': [round(avg_deep_sleep, 3)],
        'std_deep_sleep': [round(std_deep_sleep, 3)],
        'avg_total_sleep': [round(avg_total_sleep, 3)],
        'std_total_sleep': [round(std_total_sleep, 3)],
        'avg_light_sleep': [round(avg_light_sleep, 3)],
        'std_light_sleep': [round(std_light_sleep, 3)],
        'avg_NREM_sleep': [round(avg_NREM_sleep, 3)],
        'std_NREM_sleep': [round(std_NREM_sleep, 3)],
        'avg_REM_sleep': [round(avg_REM_sleep, 3)],
        'std_REM_sleep': [round(std_REM_sleep, 3)],
        'avg_awakenings': [round(avg_awakenings, 3)],
        'std_awakenings': [round(std_awakenings, 3)],
        'avg_latency': [round(avg_latency, 3)],
        'std_latency': [round(std_latency, 3)],
        'avg_efficiency': [round(avg_efficiency, 3)],
        'std_efficiency': [round(std_efficiency, 3)]
    })
    return participant_sleep_data, sleep_stats
   
def calculate_stats(data, period_mask=None):
    if period_mask is not None:
        data = data[period_mask]
        data = data.values[:,0]
    data = data[~np.isnan(data)]
    if len(data) == 0:
        return pd.DataFrame({
            'std': [np.nan],
            'min': [np.nan],
            'n5': [np.nan],
            'n25': [np.nan],
            'median': [np.nan],
            'avg': [np.nan],
            'n75': [np.nan],
            'n95': [np.nan],
            'max': [np.nan],
            'var': [np.nan],
            'std_score': [np.nan],
            'var_score': [np.nan],
            'rms': [np.nan],
            'consistency_score': [np.nan],
            'entropy': [np.nan]
        })
    max_val = round(np.nanmax(data), 3)
    min_val = round(np.nanmin(data), 3)
    avg_val = round(np.nanmean(data), 3)
    median = round(np.nanpercentile(data, 50),3)
    rms = round(np.nanmean(np.sqrt(data**2)),3)
    
    if len(data) > 1:
        n5 = round(np.nanpercentile(data, 5), 3)
        n25 = round(np.nanpercentile(data, 25), 3)
        n75 = round(np.nanpercentile(data, 75), 3)
        n95 = round(np.nanpercentile(data, 95), 3)
        std_val = round(np.nanstd(data), 3)
        var_val = round(np.nanvar(data), 3)
        std_score_val = round(100 * np.nanstd(data) / np.nanmean(data), 3)
        var_score_val = round(np.nanvar(data) / np.nanmean(data), 3)
        consistency_score_val = round(100. / np.nanvar(data), 3) if np.nanvar(data) != 0 else np.nan
        entropy = calculate_entropy(data)
    else:
        n5 = np.nan
        n25 = np.nan
        n75 = np.nan
        n95 = np.nan
        std_val = np.nan
        var_val = np.nan
        std_score_val = np.nan
        var_score_val = np.nan
        consistency_score_val = np.nan
        entropy = np.nan
    data_stats = pd.DataFrame({
        'std': [std_val],
        'min': [min_val],
        'n5': [n5],
        'n25': [n25],
        'median': [median],
        'avg': [avg_val],
        'n75': [n75],
        'n95': [n95],
        'max': [max_val],
        'var': [var_val],
        'std_score': [std_score_val],
        'var_score': [var_score_val],
        'rms': [rms],
        'consistency_score': [consistency_score_val],
        'entropy': [entropy]
    })
    return data_stats

# Helper function to extract values from the DataFrame
def extract_stats(stats_df, prefix):
    return {
        f'{prefix}_std': float(stats_df['std'].values[0]),
        f'{prefix}_min': float(stats_df['min'].values[0]),
        f'{prefix}_n5': float(stats_df['n5'].values[0]),
        f'{prefix}_n25': float(stats_df['n25'].values[0]),
        f'{prefix}_median': float(stats_df['median'].values[0]),
        f'{prefix}_avg': float(stats_df['avg'].values[0]),
        f'{prefix}_n75': float(stats_df['n75'].values[0]),
        f'{prefix}_n95': float(stats_df['n95'].values[0]),
        f'{prefix}_max': float(stats_df['max'].values[0]),
        f'{prefix}_var': float(stats_df['var'].values[0]),
        f'{prefix}_std_score': float(stats_df['std_score'].values[0]),
        f'{prefix}_var_score': float(stats_df['var_score'].values[0]),
        f'{prefix}_rms': float(stats_df['rms'].values[0]),
        f'{prefix}_consistency_score': float(stats_df['consistency_score'].values[0]),
        f'{prefix}_entropy': float(stats_df['entropy'].values[0]),
    }

def extract_prediction_stats(stats_df, prefix):
    return {
        f'{prefix}_median': float(stats_df['median'].values[0])
    }
def extract_reduced_stats(stats_df, prefix):
    return {
        f'{prefix}_median': float(stats_df['median'].values[0]),
        f'{prefix}_entropy': float(stats_df['entropy'].values[0])
    }
    
def daily_sleep_stats(daily_sleep_data):
    sleep_stats = pd.DataFrame({
        'deep_sleep': (daily_sleep_data['NVORRES'][daily_sleep_data['NVTEST'] == "Deep NREM Duration"]).values.round(2),
        'total_sleep': (daily_sleep_data['NVORRES'][daily_sleep_data['NVTEST'] == "Total Sleep Time"]).values.round(2),
        'light_sleep': (daily_sleep_data['NVORRES'][daily_sleep_data['NVTEST'] == "Light NREM Duration"]).values.round(2),
        'NREM_sleep': (daily_sleep_data['NVORRES'][daily_sleep_data['NVTEST'] == "NREM Duration"]).values.round(2),
        'REM_sleep': (daily_sleep_data['NVORRES'][daily_sleep_data['NVTEST'] == "REM Duration"]).values.round(2),
        'awakenings': (daily_sleep_data['NVORRES'][daily_sleep_data['NVTEST'] == "Number of Awakenings"]*100000).values.round(2),
        'latency': (daily_sleep_data['NVORRES'][daily_sleep_data['NVTEST'] == "Sleep Onset Latency"]).values.round(2),
        'efficiency': (daily_sleep_data['NVORRES'][daily_sleep_data['NVTEST'] == "Sleep Efficiency"]*100000).values.round(2),
        'bedtime': daily_sleep_data['bedtime_from_avg'][daily_sleep_data['NVTEST'] == "Total Sleep Time"].values.round(2),
        'wakeup': daily_sleep_data['wakeup_from_avg'][daily_sleep_data['NVTEST'] == "Total Sleep Time"].values.round(2)
    })
    return sleep_stats
def calculate_sum(carbs_data, period_mask=None):
    if period_mask is not None:
        carbs_data = carbs_data[period_mask]
    return np.sum(carbs_data['corrected_value']) if not carbs_data['corrected_value'].empty else 0

def corrected_daily_carbs_stats(daily_carb_data):
    overall_stats = calculate_stats(daily_carb_data['corrected_value'].values)
    overall_stats = extract_prediction_stats(overall_stats, 'carbs')
    
    if len(daily_carb_data) > 0:
        # Calculate carb on board at midnight considering only intake after 7 PM
        start_time = pd.Timestamp(daily_carb_data['FADTC'].dt.date.iloc[0]) + pd.Timedelta(hours=19)  # 7 PM
        midnight = pd.Timestamp(daily_carb_data['FADTC'].dt.date.iloc[0]) + pd.Timedelta(days=1)  # 12 AM next day
        carb_on_board = 0.0
        for _, row in daily_carb_data[daily_carb_data['FADTC'] >= start_time].iterrows():
            carb_amount = row['corrected_value']
            intake_time = row['FADTC']
            absorption_start_time = intake_time + pd.Timedelta(minutes=15)

            if midnight > absorption_start_time:
                absorbed_duration = min((midnight - absorption_start_time).total_seconds() / 60, carb_amount / 0.5)
                absorbed_carbs = absorbed_duration * 0.5
                remaining_carbs = max(carb_amount - absorbed_carbs, 0)
                carb_on_board += remaining_carbs
    else:
        carb_on_board = np.nan
    overall_stats['carb_on_board_midnight'] = carb_on_board
    
    masks = {
        'night': (daily_carb_data['FADTC'].dt.hour >= 0) & (daily_carb_data['FADTC'].dt.hour < 6),
        'morning': (daily_carb_data['FADTC'].dt.hour >= 6) & (daily_carb_data['FADTC'].dt.hour < 12),
        'afternoon': (daily_carb_data['FADTC'].dt.hour >= 12) & (daily_carb_data['FADTC'].dt.hour < 18),
        'evening': (daily_carb_data['FADTC'].dt.hour >= 18) & (daily_carb_data['FADTC'].dt.hour < 24),
        'late_evening': (daily_carb_data['FADTC'].dt.hour >= 21) & (daily_carb_data['FADTC'].dt.hour < 24)
    } 
    time_range_stats = {}
    if len(daily_carb_data) > 0:
        time_range_stats['carbs_day'] = calculate_sum(daily_carb_data)
    else:
        time_range_stats['carbs_day'] = np.nan
        
    for period, mask in masks.items():
        if len(daily_carb_data) > 0:
            time_range_stats[f'carbs_{period}'] = calculate_sum(daily_carb_data, mask)
        else:
            time_range_stats[f'carbs_{period}'] = np.nan

    carbs_stats = pd.DataFrame({**overall_stats, **time_range_stats}, index=[0])
    return carbs_stats

def total_stats(data, times, label_data='bolus'):
    daily_data = pd.DataFrame({'datetime': times, 'value': data})
    daily_data.set_index('datetime', inplace=True)
    
    general_stats = calculate_stats(daily_data.values[:,0])
    general_stats = extract_stats(general_stats, label_data)
    masks = {
        'night': (daily_data.index.hour >= 0) & (daily_data.index.hour < 6),
        'morning': (daily_data.index.hour >= 6) & (daily_data.index.hour < 12),
        'afternoon': (daily_data.index.hour >= 12) & (daily_data.index.hour < 18),
        'evening': (daily_data.index.hour >= 18) & (daily_data.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(daily_data, mask)
        period_stats = extract_stats(period_stats, f'{label_data}_{period}')
        time_range_stats.update(period_stats)
    
    total_stats = pd.DataFrame({**general_stats, **time_range_stats}, index=[0])
    return total_stats

def daily_stats(data, times, label_data='basal'):
    daily_data = pd.DataFrame({'datetime': times, 'value': data})
    daily_data.set_index('datetime', inplace=True)
    
    general_stats = calculate_stats(daily_data.values[:,0])
    general_stats = extract_stats(general_stats, label_data)
    mean_crossings = calculate_crossings(daily_data.values[:,0])
    general_stats[f'{label_data}_mean_crossings'] = mean_crossings
    masks = {
        'night': (daily_data.index.hour >= 0) & (daily_data.index.hour < 6),
        'morning': (daily_data.index.hour >= 6) & (daily_data.index.hour < 12),
        'afternoon': (daily_data.index.hour >= 12) & (daily_data.index.hour < 18),
        'evening': (daily_data.index.hour >= 18) & (daily_data.index.hour < 24),
        'late_evening': (daily_data.index.hour >= 22) & (daily_data.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(daily_data, mask)
        period_stats = extract_stats(period_stats, f'{label_data}_{period}')
        time_range_stats.update(period_stats)
    
    daily_stats = pd.DataFrame({**general_stats, **time_range_stats}, index=[0])
    return daily_stats
def daily_stats_bolus(data, times, label_data='bolus'):
    daily_data = pd.DataFrame({'datetime': times, 'value': data})
    daily_data.set_index('datetime', inplace=True)
    
    general_stats = calculate_stats(daily_data.values[:,0])
    general_stats = extract_stats(general_stats, label_data)
    mean_crossings = calculate_crossings(daily_data.values[:,0])
    general_stats[f'{label_data}_mean_crossings'] = mean_crossings
    masks = {
        'night': (daily_data.index.hour >= 0) & (daily_data.index.hour < 6),
        'afternoon': (daily_data.index.hour >= 12) & (daily_data.index.hour < 18),
        'evening': (daily_data.index.hour >= 18) & (daily_data.index.hour < 24)
    }
    time_range_stats = {}
    for period, mask in masks.items():
        period_stats = calculate_stats(daily_data, mask)
        period_stats = extract_stats(period_stats, f'{label_data}_{period}')
        time_range_stats.update(period_stats)
    
    daily_stats = pd.DataFrame({**general_stats, **time_range_stats}, index=[0])
    return daily_stats
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=stats.entropy(probabilities)
    return entropy.round(3)

def calculate_crossings(list_values):
    mean_crossing_indices = np.nonzero(np.diff(list_values > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return no_mean_crossings

def resample_sum(data, times, interval='5T'):
    df_data = pd.DataFrame({'datetime': times, 'value': data})
    df_data.set_index('datetime', inplace=True)
    data_resampled = df_data.copy()
    data_resampled.index = data_resampled.index.floor(interval)
    data_resampled = data_resampled.groupby(data_resampled.index).sum()
    data_resampled = data_resampled.resample(interval).asfreq()
    return data_resampled
def resample_avg(data, times, interval='5T'):
    df_data = pd.DataFrame({'datetime': times, 'value': data})
    df_data.set_index('datetime', inplace=True)
    data_resampled = df_data.copy()
    data_resampled.index = data_resampled.index.floor(interval)
    data_resampled = data_resampled.groupby(data_resampled.index).mean()
    data_resampled = data_resampled.resample(interval).asfreq()
    return data_resampled

def smooth_signal(df, window_size=17, treshold=10, order=None):
    data = df['value']

    if order is not None:
        # Temporarily fill NaNs with interpolation to avoid issues with savgol_filter
        data_filled = data.interpolate(method='linear')
        # Apply savgol_filter only to non-NaN values
        smoothed_filled = savgol_filter(data_filled.fillna(0), window_length=window_size-8, polyorder=order)
        # Reintroduce NaNs where they were originally
        smoothed_values = pd.Series(smoothed_filled, index=data.index)
        smoothed_values[np.isnan(data)] = np.nan
    else:
        smoothed_values = data
        
    half_window = window_size // 2
    # Generate a Gaussian (normal) distribution for weights
    # The mean is at the center of the window, and the standard deviation determines the spread
    weights = norm.pdf(np.arange(-half_window, half_window + 1), 0, 2)
    # Normalize weights so that they sum to 1
    weights /= weights.sum()
    data_smooth = pd.Series(smoothed_values, index=df.index)
    rolling_avg = data_smooth.rolling(window=window_size, center=True).apply(lambda x: np.dot(x, weights), raw=True)

    #rolling_avg = data.rolling(window=window_size, center=True).mean()
    fluctuations = data - rolling_avg
    smoothed_values = rolling_avg + fluctuations.where(fluctuations.abs() > treshold, 0)
    
    weights = norm.pdf(np.arange(-3, 3 + 1), 0, 2)
    weights /= weights.sum()
    rolling_avg = smoothed_values.rolling(window=7, center=True).apply(lambda x: np.dot(x, weights), raw=True)
    fluctuations = smoothed_values - rolling_avg
    smoothed_values = rolling_avg + fluctuations.where(fluctuations.abs() > treshold, 0)

    data_filled = smoothed_values.interpolate(method='linear')
    # Apply savgol_filter only to non-NaN values
    smoothed_filled = savgol_filter(data_filled.fillna(0), window_length=window_size-10, polyorder=order+1)
    # Reintroduce NaNs where they were originally
    smoothed_values = pd.Series(smoothed_filled, index=data.index)
    smoothed_values[np.isnan(data)] = np.nan
        
    smoothed_df = pd.DataFrame({'datetime': df.index, 'value': smoothed_values})
    smoothed_df.set_index('datetime', inplace=True)
    return smoothed_df

def plot_cwt(data, power_mex,period_mex,power_morl, period_morl, waveletname = 'mexh', label_data = 'CGM '):
    times = data.index
    levels = [0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8,16,32,64]
    contourlevels = np.log2(levels)
    title_mex = "MexicanHat Power Spectrum of "+ str(label_data)
    title_morl = "Morlet Power Spectrum of "+ str(label_data)
    ylabel = 'Period (Hours)'
    xlabel = 'Time'
    cmap = plt.cm.jet

    ax[0].contourf(times, np.log2(period_mex), np.log2(power_mex/1024), contourlevels, extend='both',cmap=cmap)
    ax[0].set_title(title_mex, fontsize=20)
    ax[0].set_ylabel(ylabel, fontsize=16)
    ax[0].set_xlabel(xlabel, fontsize=16)
    
    yticks = 2**np.arange(np.ceil(np.log2(period_mex.min())), np.ceil(np.log2(period_mex.max())))
    ax[0].set_yticks(np.log2(yticks))
    ax[0].set_yticklabels(yticks)
    ax[0].invert_yaxis()
    ax[0].set_ylim(np.log2([2*period_mex.min(), period_mex.max()]))
    
    ax[1].contourf(times, np.log2(period_morl), np.log2(power_morl/1024), contourlevels, extend='both',cmap=cmap)
    ax[1].set_title(title_morl, fontsize=20)
    ax[1].set_ylabel(ylabel, fontsize=16)
    ax[1].set_xlabel(xlabel, fontsize=16)
    
    yticks = 2**np.arange(np.ceil(np.log2(period_morl.min())), np.ceil(np.log2(period_morl.max())))
    ax[1].set_yticks(np.log2(yticks))
    ax[1].set_yticklabels(yticks)
    ax[1].invert_yaxis()
    ax[1].set_ylim(np.log2([2*period_morl.min(), period_morl.max()]))
    
    if len(data)<300:
        ax[0].xaxis.set_major_locator(mdates.HourLocator(interval=2))
        ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%m/%d--%H:%M'))
        ax[0].xaxis.set_minor_locator(mdates.HourLocator(byhour=[6, 12, 18]))
        ax[0].xaxis.grid(True, which='minor') 

        ax[1].xaxis.set_major_locator(mdates.HourLocator(interval=2))
        ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%m/%d--%H:%M'))
        ax[1].xaxis.set_minor_locator(mdates.HourLocator(byhour=[6, 12, 18]))
        ax[1].xaxis.grid(True, which='minor') 
    else:
        ax[0].xaxis.set_major_locator(mdates.DayLocator(interval=1))
        ax[0].xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
        ax[1].xaxis.set_major_locator(mdates.DayLocator(interval=1))
        ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
        
    return ax[0],ax[1]

def compute_cwt(data, scales, dt=1./12, normalize = True, waveletname = 'mexh', label_data = 'CGM ' ):
    times = data.index
    N = len(data)
    if waveletname in ['cmor', 'shan']:
        waveletname += '1.5-1'
      
    if normalize:
        data_detrended = (data - data.mean())/data.std()
    else:
        data_detrended = data #- data.mean()
        var = np.var(data_detrended).iloc[0]
    
    [coefficients_mex, frequencies_mex] = pywt.cwt(data_detrended.values.squeeze(), scales, waveletname, dt)
    [coefficients_morl, frequencies_morl] = pywt.cwt(data_detrended.values.squeeze(), 4*scales, 'morl', dt)
    period_mex = 1. / frequencies_mex
    period_morl = 1. / frequencies_morl
    
    fft = np.fft.fft(data_detrended)
    fftfreqs = np.fft.fftfreq(N, dt)
    fft_power = np.abs(fft) ** 2
    scaleMatrix = np.ones([1, N]) * scales[:, None]

    if normalize:
        power_mex =  (abs(coefficients_mex)) ** 2 / scaleMatrix
    else:
        power_mex  = (abs(coefficients_mex)) ** 2 / (scaleMatrix) 
        
    if normalize:
        power_morl =  (abs(coefficients_morl)) ** 2 / scaleMatrix
    else:
        power_morl  = (abs(coefficients_morl)) ** 2 / (scaleMatrix) 
    
    signed_log_power_mex = np.sign(coefficients_mex) * np.log2(power_mex)
    signed_log_power_morl = np.sign(coefficients_morl) * np.log2(power_morl)
    # Replace -inf with the minimum finite value
    signed_log_power_mex = np.where(np.isfinite(signed_log_power_mex), signed_log_power_mex, np.min(signed_log_power_mex[np.isfinite(signed_log_power_mex)])/10)
    signed_log_power_morl = np.where(np.isfinite(signed_log_power_morl), signed_log_power_morl, np.min(signed_log_power_morl[np.isfinite(signed_log_power_morl)])/10)

    # ax[0], ax[1] = plot_cwt(data, power_mex,period_mex,power_morl, period_morl, waveletname = waveletname, label_data = label_data )
    glbl_power = power_mex.mean(axis=1)
    #ax[0],ax[1], 
    return coefficients_mex,power_mex,signed_log_power_mex,period_mex,coefficients_morl,power_morl,signed_log_power_morl,period_morl, fft, fftfreqs, fft_power, glbl_power

def plot_loss_accuracy(history):
    plt.figure(figsize=(12, 5))

    # Plot loss
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    # Plot accuracy
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

def remove_nan_rows(X_daily, X_individual, y_label):
    # Find rows with NaNs in either feature vector
    nan_mask_daily = np.any(np.isnan(X_daily), axis=(1, 2))
    nan_mask_individual = np.any(np.isnan(X_individual), axis=(1, 2))
    
    nan_mask_combined = nan_mask_daily | nan_mask_individual    
    # Keep only rows without NaNs
    X_daily_clean = X_daily[~nan_mask_combined]
    X_individual_clean = X_individual[~nan_mask_combined]
    y_label_clean = y_label[~nan_mask_combined]
    return X_daily_clean, X_individual_clean, y_label_clean

def apply_smote_resampling(X_matrices, X_features_daily, X_features_individual, y, matrix_shape0=288):
    smote = SMOTE(random_state=42)

    # Combine all inputs into a single 2D array for SMOTE
    X_combined = np.hstack([
        X_matrices.reshape(-1, matrix_shape0 * 288*3),
        X_features_daily,
        X_features_individual
    ])

    # Apply SMOTE to the combined array and the target
    X_resampled_combined, y_resampled = smote.fit_resample(X_combined, y)

    # Extract the resampled components
    X_matrices_resampled = X_resampled_combined[:, :matrix_shape0 * 288*3].reshape(-1, matrix_shape0, 288, 3)
    start = matrix_shape0 * 288*3
    X_features_daily_resampled = X_resampled_combined[:, start:start + X_features_daily.shape[1]]
    start += X_features_daily.shape[1]
    X_features_individual_resampled = X_resampled_combined[:, start:]
    
    return (X_matrices_resampled, 
            X_features_daily_resampled, 
            X_features_individual_resampled, 
            y_resampled)
def apply_smote_resampling_mask(X_matrices, X_features_daily, X_features_individual, mask_daily, mask_individual, y, matrix_shape0=288):
    smote = SMOTE(random_state=42)
    X_combined = np.hstack([
        X_matrices.reshape(-1, matrix_shape0 * 288*3),
        X_features_daily,
        X_features_individual,
        mask_daily,
        mask_individual
    ])
    X_resampled_combined, y_resampled = smote.fit_resample(X_combined, y)

    X_matrices_resampled = X_resampled_combined[:, :matrix_shape0 * 288*3].reshape(-1, matrix_shape0, 288, 3)
    start = matrix_shape0 * 288*3
    X_features_daily_resampled = X_resampled_combined[:, start:start + X_features_daily.shape[1]]
    start += X_features_daily.shape[1]
    X_features_individual_resampled = X_resampled_combined[:, start:start + X_features_individual.shape[1]]
    start += X_features_individual.shape[1]
    mask_daily_resampled = X_resampled_combined[:, start:start + mask_daily.shape[1]]
    start += mask_daily.shape[1]
    mask_individual_resampled = X_resampled_combined[:, start:]

    return (X_matrices_resampled, 
            X_features_daily_resampled, 
            X_features_individual_resampled, 
            mask_daily_resampled, 
            mask_individual_resampled, 
            y_resampled)

def apply_smote_resampling_mask_2d(X_matrices, X_features_daily, X_features_individual, mask_daily, mask_individual, y, matrix_shape0=288):
    smote = SMOTE(random_state=42)
    X_combined = np.hstack([
        X_matrices.reshape(-1, matrix_shape0 * 288*2),
        X_features_daily,
        X_features_individual,
        mask_daily,
        mask_individual
    ])
    X_resampled_combined, y_resampled = smote.fit_resample(X_combined, y)

    X_matrices_resampled = X_resampled_combined[:, :matrix_shape0 * 288*2].reshape(-1, matrix_shape0, 288, 2)
    start = matrix_shape0 * 288*2
    X_features_daily_resampled = X_resampled_combined[:, start:start + X_features_daily.shape[1]]
    start += X_features_daily.shape[1]
    X_features_individual_resampled = X_resampled_combined[:, start:start + X_features_individual.shape[1]]
    start += X_features_individual.shape[1]
    mask_daily_resampled = X_resampled_combined[:, start:start + mask_daily.shape[1]]
    start += mask_daily.shape[1]
    mask_individual_resampled = X_resampled_combined[:, start:]

    return (X_matrices_resampled, 
            X_features_daily_resampled, 
            X_features_individual_resampled, 
            mask_daily_resampled, 
            mask_individual_resampled, 
            y_resampled)
    


In [10]:
data_dict = defaultdict(lambda: defaultdict(dict))

for participant_id in shared_participants: 
    participant_insulin_data = T1DEXI_basal[T1DEXI_basal['USUBJID'] == participant_id].reset_index()     
    participant_cgm_data = T1DEXI_CGM_Dataset[T1DEXI_CGM_Dataset['USUBJID'] == participant_id].reset_index()
    # participant_sleep_data = T1DEXI_sleep_Dataset[T1DEXI_sleep_Dataset['USUBJID'] == participant_id].reset_index()
    participant_hr_data = T1DEXI_VS_Dataset[T1DEXI_VS_Dataset['USUBJID'] == participant_id].reset_index()

    participant_Reqcue_Carbs = T1DEXI_Reqcue_Carbs_Dataset[T1DEXI_Reqcue_Carbs_Dataset['USUBJID'] == participant_id][T1DEXI_Reqcue_Carbs_Dataset['MLCAT']=='RESCUE CARBS']
    participant_carbs_self = FAMLPM_Dataset[FAMLPM_Dataset['USUBJID']==participant_id][FAMLPM_Dataset['FATESTCD']=='DCARBT'][FAMLPM_Dataset['FACAT']=='CONSUMED'][FAMLPM_Dataset['FAMETHOD']=='SELF-REPORT'].reset_index()
    participant_carbs_RFPM = FAMLPM_Dataset[FAMLPM_Dataset['USUBJID']==participant_id][FAMLPM_Dataset['FATESTCD']=='DCARBT'][FAMLPM_Dataset['FACAT']=='CONSUMED'][FAMLPM_Dataset['FAMETHOD']=='RFPM'].reset_index()

    merged_carbs = pd.merge(participant_carbs_self[['FAORRES','FADTC']], participant_carbs_RFPM[['FAORRES','FADTC']], on='FADTC', how='left', suffixes=('_self', '_RFPM'))
    merged_carbs['corrected_value'] = (merged_carbs['FAORRES_self'] + merged_carbs['FAORRES_RFPM']) / 2
    participant_carbs_self['corrected_value'] = merged_carbs['corrected_value'].fillna(participant_carbs_self['FAORRES'])        
    participant_carbs_self = participant_carbs_self.dropna(subset=['corrected_value']).reset_index()
    participant_rescue_carbs_renamed = participant_Reqcue_Carbs.rename(columns={
        'MLDTC': 'FADTC',
        'MLDOSE': 'corrected_value',
        'MLCAT': 'FAOBJ'
    }).dropna(subset=['corrected_value']).reset_index()
    participant_carbs = pd.concat([participant_carbs_self[['FADTC', 'corrected_value', 'FAOBJ']], participant_rescue_carbs_renamed[['FADTC', 'corrected_value', 'FAOBJ']]], ignore_index=True).sort_values(by='FADTC')
    if participant_hr_data.empty:
        continue
    a1c_data = participant_cgm_data[participant_cgm_data['LBTEST'] == "Hemoglobin A1c"]
    a1c = a1c_data['LBSTRESN'].iloc[0]
    height_data = participant_hr_data[participant_hr_data['VSTESTCD'] == "HEIGHT"]
    height = height_data['VSSTRESN'].iloc[0]
    weight_data = participant_hr_data[participant_hr_data['VSTESTCD'] == "WEIGHT"]
    weight = weight_data['VSSTRESN'].iloc[0]
    print("\nparticipant id: ", participant_id)

    # --- Separate data types ---
    participant_basal_data = participant_insulin_data[participant_insulin_data['FACAT'] == "BASAL"][participant_insulin_data['FATESTCD'] == "INSBASAL"].dropna(subset=['FASTRESN']).reset_index() #BASFLRT  #INSBASAL
    participant_bolus_data = participant_insulin_data[participant_insulin_data['FACAT'] == "BOLUS"][participant_insulin_data['FATESTCD'] == "INSBOLUS"].dropna(subset=['FASTRESN']).reset_index() 
    participant_cgm_data = participant_cgm_data[participant_cgm_data['LBTEST'] == "Glucose"].dropna(subset=['LBSTRESN']).reset_index()
    participant_hr_data = participant_hr_data[participant_hr_data['VSTESTCD']=='HRM'].dropna(subset=['VSSTRESN']).reset_index()

    # --- Align data to 5-minute intervals ---    
    aligned_cgm = resample_avg(participant_cgm_data['LBSTRESN'], participant_cgm_data['LBDTC'], interval='5T')
    # --- Interpolate max 3 hour gaps ---    
    method = 'linear'  
    interpolated_cgm = aligned_cgm.interpolate(method=method, order=2,limit_direction='both')
    
    smoothed_cgm = smooth_signal(interpolated_cgm, window_size=17, treshold=10, order=2)
    interpolated_cgm = smoothed_cgm.interpolate(method=method, order=2,limit_direction='both')
  
    # Total Participant Stats
    total_cgm_stats = get_total_cgm_stats(aligned_cgm)
    total_bolus_stats = total_stats(participant_bolus_data['FASTRESN'], participant_bolus_data['FADTC'], 'bolus')
    total_basal_stats = total_stats(participant_basal_data['FASTRESN'], participant_basal_data['FADTC'], 'basal')
    total_hr_stats = total_stats(participant_hr_data['VSSTRESN'], participant_hr_data['VSDTC'], 'hr')

# --- Compute & Plot overall CWT ---    
    dt_cgm = 1./12 
    s0 = 1./2  #starting period is 15 minutes
    dj = 1/8  # Resolusion
    J = int(12 / dj)
    #scales = s0 * 2 ** (dj * np.arange(0, J + 1))
    scales = s0 * np.arange(1,576,2)

    # fig, ax = plt.subplots(2, 1, figsize=(20, 10), sharex=True)  
    #ax[0], ax[1],
    coefficients_mex,power_mex,signed_log_power_mex,period_mex,coefficients_morl,power_morl,signed_log_power_morl,period_morl, fft, fftfreqs, fft_power_cgm, glbl_power_cgm = compute_cwt(interpolated_cgm, scales, dt=dt_cgm , normalize = False, waveletname = 'mexh', label_data = "CGM ID#" +str(participant_id) )
 
    # --- Find overlapping dates ---
    unique_dates = sorted(set(interpolated_cgm.index.date)) #& set(participant_carbs['FADTC'].dt.date) & set(participant_bolus_data['FADTC'].dt.date) & set(participant_sleep_data['NVENDTC'].dt.date) & set(participant_basal_data['FADTC'].dt.date) & set(participant_hr_data['VSDTC'].dt.date)
    unique_dates = unique_dates[1:-1] # Skip the first & last days
    for current_date in unique_dates:
        # --- Get current day data (for feature extraction) ---
        current_day_mask_cgm = interpolated_cgm.index.date == current_date
        current_day_indices_cgm = np.where(current_day_mask_cgm)[0]
        
        current_day_basal = participant_basal_data[participant_basal_data['FADTC'].dt.date == current_date]
        current_day_bolus = participant_bolus_data[participant_bolus_data['FADTC'].dt.date == current_date]
        current_day_mask_cgm = current_day_mask_cgm.astype(bool)
        current_day_cgm = interpolated_cgm[current_day_mask_cgm]
        current_day_cgm_original = aligned_cgm[aligned_cgm.index.date == current_date]
        current_day_hr = participant_hr_data[participant_hr_data['VSDTC'].dt.date == current_date]
        current_day_carbs = participant_carbs[participant_carbs['FADTC'].dt.date == current_date]
        if current_day_cgm_original.notna().sum()[0]<252 or current_day_cgm_original[-24:].notna().sum()[0]<18:
            continue 

        current_day_carbs_stats = corrected_daily_carbs_stats(current_day_carbs)   
        current_day_cgm_stats = get_daily_cgm_stats(current_day_cgm)
        current_day_cgm_stats_original = get_daily_cgm_stats(current_day_cgm_original)  
        
        current_day_bolus_stats = daily_stats_bolus(current_day_bolus['FASTRESN'], current_day_bolus['FADTC'], 'bolus_daily')
        current_day_basal_stats = daily_stats(current_day_basal['FASTRESN'], current_day_basal['FADTC'], 'basal_daily')
#         current_day_basal_stats = daily_stats(current_day_basal.values.squeeze(), current_day_basal.index, 'basal_daily')
        current_day_hr_stats = daily_stats(current_day_hr['VSSTRESN'], current_day_hr['VSDTC'], 'hr_daily')
        
        next_date = current_date + pd.Timedelta(days=1) # Get one hour before 
        next_day_cgm = interpolated_cgm[interpolated_cgm.index.date == next_date]
        next_day_cgm_original = aligned_cgm[aligned_cgm.index.date == next_date]
        next_night = current_day_cgm.index[-1] + pd.Timedelta(hours=8) #More than 2hour missing in the next 6 hours
        if np.sum(next_day_cgm_original.index <= next_night)<72:
            continue 
        # next_day_8pm = next_day_cgm_original[next_day_cgm_original.index <= next_night]
        # if next_day_8pm.isna().sum()[0]>24:
        #     continue
            
        next_day_cgm_stats = cgm_prediction_stats(next_day_cgm)
        next_day_cgm_labels = daily_cgm_labels(next_day_cgm)
        next_day_cgm_stats_original = cgm_prediction_stats(next_day_cgm_original)
        next_day_cgm_labels_original = daily_cgm_labels(next_day_cgm_original)
        
        # --- for Plotting 48 hrs, the current and previous day ---
        start_time = current_day_cgm.index[0] 
        end_time = current_day_cgm.index[-1] + pd.Timedelta(days=1)

        original_basal = participant_basal_data[(participant_basal_data['FADTC']>=start_time)&(participant_basal_data['FADTC']<=end_time)]
        original_bolus = participant_bolus_data[(participant_bolus_data['FADTC']>=start_time)&(participant_bolus_data['FADTC']<=end_time)]
        original_cgm = participant_cgm_data[(participant_cgm_data['LBDTC']>=start_time)&(participant_cgm_data['LBDTC']<=end_time)] 
        original_hr = participant_hr_data[(participant_hr_data['VSDTC']>=start_time)&(participant_hr_data['VSDTC']<=end_time)] 
        data_carbs = participant_carbs[(participant_carbs['FADTC']>= start_time) & (participant_carbs['FADTC']<=end_time)]
#         data_sleep = participant_sleep_data_updated[(participant_sleep_data_updated['NVDTC']>= start_time) & (participant_sleep_data_updated['NVENDTC']<=end_time)]        
        matrix_shape0 = len(scales)
        matrix_shape1 = len(current_day_cgm)
        daily_power_mex = power_mex[:, current_day_indices_cgm]
        daily_signed_log_power_mex = signed_log_power_mex[:, current_day_indices_cgm]
        daily_coeffs_mex = coefficients_mex[:, current_day_indices_cgm]
        daily_signed_log_power_morl = signed_log_power_morl[:, current_day_indices_cgm]
        daily_power_morl = power_morl[:, current_day_indices_cgm]
        daily_coeffs_morl = coefficients_morl[:, current_day_indices_cgm]
        
        full_matrix1 = np.ndarray(shape=(matrix_shape0, matrix_shape1,3))
        full_matrix1[:, :, 0] = daily_coeffs_mex
        full_matrix1[:, :, 1] = daily_power_mex
        full_matrix1[:, :, 2] = daily_power_morl
        full_matrix2 = np.ndarray(shape=(matrix_shape0, matrix_shape1,3))
        full_matrix2[:, :, 0] = daily_coeffs_morl
        full_matrix2[:, :, 1] = daily_signed_log_power_morl
        full_matrix2[:, :, 2] = daily_signed_log_power_mex
        
            # Store data in the dictionary
        data_dict[participant_id][current_date] = {
            'len_original_cgm': len(current_day_cgm_original),
            'full_matrix_cgm_power': full_matrix1,
            'full_matrix_cgm_coeffs': full_matrix2,
            
            'next_day_cgm_stats':next_day_cgm_stats,
            'next_day_cgm_labels':next_day_cgm_labels,
            'CGM_stats_daily': current_day_cgm_stats,
            'next_day_cgm_stats_original':next_day_cgm_stats_original,
            'next_day_cgm_labels_original':next_day_cgm_labels_original,
            'CGM_stats_daily_original': current_day_cgm_stats_original,
            
            'Basal_stats_daily': current_day_basal_stats,
            'Carbs_stats_daily': current_day_carbs_stats,
#             'Sleep_stats_daily': current_day_sleep_stats[:1],
            'HR_stats_daily': current_day_hr_stats,
            
            'A1c': a1c,
            'Weight': weight,
            'Height': height,
            'CGM_stats_participant': total_cgm_stats,
            'Basal_stats_participant': total_basal_stats,
#             'Sleep_stats_participant': sleep_stats,
            'HR_stats_participant': total_hr_stats
        }
       


participant id:  1

participant id:  4

participant id:  11

participant id:  14

participant id:  18

participant id:  24

participant id:  25

participant id:  29

participant id:  32

participant id:  34

participant id:  37

participant id:  43

participant id:  48

participant id:  50

participant id:  54

participant id:  58

participant id:  60

participant id:  63

participant id:  66

participant id:  67

participant id:  69

participant id:  71

participant id:  77

participant id:  78

participant id:  80

participant id:  84

participant id:  85

participant id:  95

participant id:  97

participant id:  103

participant id:  104

participant id:  107

participant id:  110

participant id:  114

participant id:  115

participant id:  127

participant id:  137

participant id:  143

participant id:  144

participant id:  145

participant id:  152

participant id:  159

participant id:  163

participant id:  166

participant id:  167

participant id:  173

participant id:  1

In [14]:
filtered_data_dict = {}
for participant_id, days_data in data_dict.items():
    filtered_days_data = []
    for current_date, day_data in days_data.items():

        if (day_data['len_original_cgm'] > 252 and np.sum(day_data['CGM_stats_daily_original'].isna()).sum()<1):
            filtered_days_data.append(day_data)
    
    if filtered_days_data:
        filtered_data_dict[participant_id] = filtered_days_data

print(len(data_dict),len(filtered_data_dict))


491 491


In [12]:

with open('/mnt/data1/ma98/data_generated/dict_3d_cgm_mex_morl_smooth_70_15min.dill', 'wb') as f:
    dill.dump(data_dict, f)



In [13]:
with open('/mnt/data1/ma98/data_generated/dict_3d_cgm_mex_morl_smooth_70_15min.dill', 'rb') as f:
    loaded_dict = dill.load(f)
    
data_dict = loaded_dict.copy()
print(len(data_dict))


491


In [15]:
with open('/home/ma98/temp/dict_3d_cgm_mex_morl_smooth_70_15min.dill', 'wb') as f:
    dill.dump(data_dict, f)
